# Calcular inversión por plataforma

Se calculan con las plataformas de **Meta** y **Google Ads**

### Paso a paso:
* **Lecutra y clasificación de la marca de cada anuncio**: Cada anuncio tiene en su estructura de nombre, a qué marca se invirtió. La idea es poder generar otra columna que permita clasificar cada una de las marcas que se encuentren en el archivo.

* **Inversión de cada marca según la semana de las plataformas**: Calculo de como se ha invertido el dinero en cada una de las marcas según la semana de cada una de las plataformas.

* **Distribución del costo de la Genérica con las marcas**: Cada semana se tiene una inversión entre marcas y genérica, de la genérica, se tiene que distribuir el costo entre cada una de las marcas que tenemos en la PLP, primera página.

* **Calcular los costos por RQLs**: Teniendo los costos de cada una de las plataformas, se calcula los costos por RQL, es decir, cuanto costó un RQL.

In [21]:
import sys
sys.path.append('../')
import pandas as pd
from utils.classify_ads import *
from utils.df_utils import set_week_and_year
from plataforms.utils.calculate import *
from datetime import datetime

# Procesamiento de datos
from plataforms.utils.process_data import get_value_counts_per_brand

# Obtener los datos generales de las publicaciones através de la BD de mongodb
from utils.mongodb_call import get_data_general

## Lectura y clasificación de la marca de cada anuncio

#### Meta

In [22]:
# Lectura del archivo de Meta
df_meta_ads = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\BD Ads\Abril\14042025\MX BD Ads saturación mensual - Meta.csv")

# Aplicar la función a la columna Ad name
df_meta_ads['Marca'] = df_meta_ads['Ad name'].apply(classify_ads)

# Agregar columnas con el número de semana y año
df_meta_ads = set_week_and_year(df_meta_ads)

#### Google Ads

In [23]:
# Lectura del archivo de Google Ads
df_google_ads = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\BD Ads\Abril\14042025\MX BD Ads saturación mensual - GAds.csv")

# Aplicar la función a la columna Ad name
df_google_ads['Marca'] = df_google_ads['Ad group name'].apply(classify_ads)

# Agregar columnas con el número de semana y año
df_google_ads = set_week_and_year(df_google_ads)

## Inversión de cada marca por semana de cada plataforma

In [24]:
#
# todo: Agregar la inversión de TikTok

Calculo de la inversión que se realizó en la plataforma de Meta y Google Ads

In [25]:
# Calculo de inversión por semana y año de ambas plataformas
df_meta_ads = calculate_invest_per_week(df_meta_ads)
df_google_ads = calculate_invest_per_week(df_google_ads)

# Inversión total por semana y año de ambas plataformas
invest_total = calculate_invest_from_plataforms_per_brand(df_meta_ads, df_google_ads, show_result = True)

,Marca,Semana,Año,Cost
31,Generica,1,2025,97361.79
0,Bajaj,1,2025,58725.58
47,Honda,1,2025,30336.70
78,MB,1,2025,12291.64
117,Vento,1,2025,10027.66
...,...,...,...,...
77,Italika,16,2025,134.21
62,Honda,16,2025,127.45
97,Suzuki,16,2025,21.35
30,Dinamo,16,2025,3.85


## Distribución del costo de la Genérica con las marcas

### Distribución PLP Desktop

Acá queremos saber como se distribuye el dinero invertido en cada una de las marcas. Para eso, se tiene que saber cuales son los modelos que tenemos listados en el marketplace (PLP - primera página) y distribuir los costos de la genérica entre cada una de las marcas

In [26]:
# Data con la información de los modelos en las primeras 18 posiciones
df_data_mx = get_data_general()

# Cantidad de modelos por marca listados en la primera página del marketplace
df_counts_per_brand, total_models = get_value_counts_per_brand(df_data_mx)
df_counts_per_brand

,brand,count
0,Bajaj,11
1,Vento,4
2,Honda,1
3,Italika,1
4,TVS,1


## Semana a consultar

In [27]:
week = 15
actual_year = datetime.now().year

In [28]:
# Inversión de la genérica en la semana 14
generic_invest = calculate_generic_invest(invest_total, week)

# Distribución del costo de la genérica entre las marcas
df_total_distribution = calculate_generic_invest_distribution(df_counts_per_brand, generic_invest, total_models)

Inversión genérica semana 15 es: 204726


Tabla de costos sin columna de total distribuido por marca

In [29]:
df_total_distribution

,brand,count,Cost
0,Bajaj,11,125110.33
1,Vento,4,45494.67
2,Honda,1,11373.67
3,Italika,1,11373.67
4,TVS,1,11373.67
5,Total,18,204726.00


### Distribución de costos de la genérica con las marcas

In [30]:
df_distribute_invest = distribute_generic_invest(invest_total, 
                                                 week, 
                                                 df_total_distribution)

# Se le agrega la semana y año a la distribución
df_distribute_invest['Semana'] = week
df_distribute_invest['Año'] = actual_year
df_distribute_invest

c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\plataforms\utils\calculate_utils.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['brand_categoria'] = df['brand'].apply(categorizar_marca)


,brand_categoria,original_cost,distribute_cost,total_cost,Semana,Año
0,Bajaj,154861.19,125110.33,279971.52,15,2025
1,Vento,13715.32,45494.67,59209.99,15,2025
2,Honda,17842.59,11373.67,29216.26,15,2025
3,Otros,23448.56,0.00,23448.56,15,2025
4,Italika,9690.80,11373.67,21064.47,15,2025
5,TVS,9662.98,11373.67,21036.65,15,2025
6,Total,229221.44,204726.01,433947.45,15,2025


## Calcular los costos por RQLs

Agrupar RQL por marca 

In [211]:
rqls_por_marca = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\invest_per_state_and_rql_counts\RQLs Amplitude\20250414-rql_resumen_per_brand.csv")

# Semana a consultar
rqls_por_marca_week = rqls_por_marca[rqls_por_marca["Semana"] == week]
rqls_por_marca_week_grouped = rqls_por_marca_week.groupby(['brand', 'Semana', 'Año'])['rql'].sum().reset_index()

In [212]:
rqls_por_marca_week_grouped.head()

,brand,Semana,Año,rql
0,Bajaj,15,2025,1927.0
1,Benelli,15,2025,63.0
2,CF Moto,15,2025,507.0
3,CFLITE,15,2025,112.0
4,Carabela,15,2025,4.0


In [213]:
print(rqls_por_marca_week_grouped["rql"].sum())

4851.0


Se selecciona la inversión de la semana

In [214]:
invest_total_week =  invest_total[invest_total["Semana"] == week]
invest_total_week

,Marca,Semana,Año,Cost
45,Generica,15,2025,204726.93
14,Bajaj,15,2025,154861.19
61,Honda,15,2025,17842.59
147,Yamaha,15,2025,13920.50
131,Vento,15,2025,13715.32
76,Italika,15,2025,9690.80
112,TVS,15,2025,9662.98
96,Suzuki,15,2025,8594.00
29,Dinamo,15,2025,773.20
115,Treck,15,2025,160.86


Para entender de donde viene la marca (ya sea por inversión o RQLs) se renombra

In [215]:
invest_total_week_copy = invest_total_week.copy()
invest_total_week_copy.rename(columns={"Marca": "brand_invest"}, inplace=True)

rqls_por_marca_week_grouped_copy = rqls_por_marca_week_grouped.copy()
rqls_por_marca_week_grouped_copy.rename(columns={"brand": "brand_rqls"}, inplace=True)

df_invest_rql_cost_per_brand = pd.merge(invest_total_week_copy, rqls_por_marca_week_grouped_copy, 
                                        left_on=['brand_invest', 'Semana', 'Año'], 
                                        right_on=['brand_rqls', 'Semana', 'Año'], 
                                        how='outer')
df_invest_rql_cost_per_brand

,brand_invest,Semana,Año,Cost,brand_rqls,rql
0,Bajaj,15,2025,154861.19,Bajaj,1927.0
1,NaN,15,2025,NaN,Benelli,63.0
2,NaN,15,2025,NaN,CF Moto,507.0
3,NaN,15,2025,NaN,CFLITE,112.0
4,NaN,15,2025,NaN,Carabela,4.0
5,Dinamo,15,2025,773.20,Dinamo,20.0
6,Generica,15,2025,204726.93,NaN,NaN
7,NaN,15,2025,NaN,Hero,60.0
8,Honda,15,2025,17842.59,Honda,315.0
9,NaN,15,2025,NaN,Husqvarna,9.0


### Marcas a las que se les invierte en anuncios

In [216]:
marcas_invertidas = df_invest_rql_cost_per_brand[df_invest_rql_cost_per_brand["brand_invest"].notna()]
total_rqls_marcas_invertidas = marcas_invertidas["rql"].sum()
print("Total de RQLs de las marcas invertidas: ", total_rqls_marcas_invertidas)
display(marcas_invertidas)

Total de RQLs de las marcas invertidas:  3755.0


,brand_invest,Semana,Año,Cost,brand_rqls,rql
0,Bajaj,15,2025,154861.19,Bajaj,1927.0
5,Dinamo,15,2025,773.20,Dinamo,20.0
6,Generica,15,2025,204726.93,NaN,NaN
8,Honda,15,2025,17842.59,Honda,315.0
10,Italika,15,2025,9690.80,Italika,368.0
17,Suzuki,15,2025,8594.00,Suzuki,192.0
18,TVS,15,2025,9662.98,TVS,112.0
19,Treck,15,2025,160.86,Treck,2.0
20,Vento,15,2025,13715.32,Vento,625.0
21,Yamaha,15,2025,13920.50,Yamaha,194.0


### Marcas a las que NO se les invierte en anuncios

In [217]:
marcas_no_invertidas = df_invest_rql_cost_per_brand[df_invest_rql_cost_per_brand["brand_invest"].isna()]
total_rqls_marcas_no_invertidas = marcas_no_invertidas["rql"].sum()
print("Total de RQLs de las marcas no invertidas: ", total_rqls_marcas_no_invertidas)
display(marcas_no_invertidas)

Total de RQLs de las marcas no invertidas:  1096.0


,brand_invest,Semana,Año,Cost,brand_rqls,rql
1,NaN,15,2025,NaN,Benelli,63.0
2,NaN,15,2025,NaN,CF Moto,507.0
3,NaN,15,2025,NaN,CFLITE,112.0
4,NaN,15,2025,NaN,Carabela,4.0
7,NaN,15,2025,NaN,Hero,60.0
9,NaN,15,2025,NaN,Husqvarna,9.0
11,NaN,15,2025,NaN,Jiajue,10.0
12,NaN,15,2025,NaN,KTM,191.0
13,NaN,15,2025,NaN,Kawasaki,80.0
14,NaN,15,2025,NaN,Kymco,1.0


Confirmar cantidad de RQLs entre las marcas que no se les invierte vs las que si se les invierte

In [218]:
total_rqls_entre_marcas_invertidas_y_no_invertidas = total_rqls_marcas_invertidas + total_rqls_marcas_no_invertidas
print("Total de RQLs entre las marcas invertidas y no invertidas: ", total_rqls_entre_marcas_invertidas_y_no_invertidas)

Total de RQLs entre las marcas invertidas y no invertidas:  4851.0


## Costo de RQLs por marca

#### Distribución de costos de la genérica

In [219]:
# Se copia el dataframe
df_total_distribution_copy = df_total_distribution.copy()


# Eliminar fila del "total"
df_total_distribution_copy = df_total_distribution_copy.iloc[:-1]

# Se unen las columnas de inversión de las marcas con la distribución de la genérica para realizar la suma
df_costo_distribuido_entre_marcas = pd.merge(marcas_invertidas, df_total_distribution_copy,  left_on="brand_invest", right_on="brand", how="outer")

# Se rellena con 0 los valores nulos para que la suma sea correcta
df_costo_distribuido_entre_marcas["Cost_y"].fillna(0, inplace=True)

# Se realiza la suma de las dos columnas de inversión
df_costo_distribuido_entre_marcas['costo_distribuido'] = df_costo_distribuido_entre_marcas['Cost_x'] + df_costo_distribuido_entre_marcas['Cost_y']

# Se eliminan las columnas que se utilizaron para la suma y mantenemos la del costo distribuido
df_costo_distribuido_entre_marcas.drop(columns=["Cost_x", "Cost_y", "brand", "count"], inplace=True)

C:\Users\JTRUJILLO\AppData\Local\Temp\ipykernel_11732\3971915918.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_costo_distribuido_entre_marcas["Cost_y"].fillna(0, inplace=True)


## Reporte

### RQLs

In [193]:
def group_other_brands(df, main_brands):
    """
    Agrupa las marcas que no son principales en "Otro"
    """
    marcas_no_principales = df[~df["brand"].isin(main_brands)]
    if not marcas_no_principales.empty:
        costo_otros = marcas_no_principales["rql"].sum()
        # Eliminar las marcas no principales
        df = df[df["brand"].isin(main_brands)]

        # Agregar la fila "Otro" con la suma de los costos
        otro_registro = pd.DataFrame({"brand": ["Otro"], "rql": [costo_otros]})
        df = pd.concat([df, otro_registro], ignore_index=True)

    return df

In [194]:
def generar_tabla_final_marcas(df):

    main_brands = get_main_brands()

    # Se agrupan las marcas que no son principales
    df_validated = group_other_brands(df, main_brands)

    return df_validated

In [195]:
rqls_por_marca_tested = rqls_por_marca.drop(columns = ["Semana", "Año"])
rqls_por_marca_tested["rql"] = rqls_por_marca_tested["rql"].astype(int)

In [196]:
rqls_marca_final = generar_tabla_final_marcas(rqls_por_marca_tested)

In [197]:
rqls_marca_final

,brand,rql
0,Bajaj,1927
1,CF Moto,507
2,Honda,315
3,Italika,368
4,Suzuki,192
5,TVS,112
6,Vento,625
7,Yamaha,194
8,Otro,611


In [198]:
print(rqls_marca_final["rql"].sum())

4851


In [31]:
df_distribute_invest

,brand_categoria,original_cost,distribute_cost,total_cost,Semana,Año
0,Bajaj,154861.19,125110.33,279971.52,15,2025
1,Vento,13715.32,45494.67,59209.99,15,2025
2,Honda,17842.59,11373.67,29216.26,15,2025
3,Otros,23448.56,0.00,23448.56,15,2025
4,Italika,9690.80,11373.67,21064.47,15,2025
5,TVS,9662.98,11373.67,21036.65,15,2025
6,Total,229221.44,204726.01,433947.45,15,2025


In [39]:
invest_total_week

NameError: name 'invest_total_week' is not defined

In [33]:
df_distribute_invest_copy = df_distribute_invest.copy()


In [35]:
df_distribute_invest_copy

,brand_categoria,original_cost,distribute_cost,total_cost,Semana,Año
0,Bajaj,154861.19,125110.33,279971.52,15,2025
1,Vento,13715.32,45494.67,59209.99,15,2025
2,Honda,17842.59,11373.67,29216.26,15,2025
3,Otros,23448.56,0.00,23448.56,15,2025
4,Italika,9690.80,11373.67,21064.47,15,2025
5,TVS,9662.98,11373.67,21036.65,15,2025
6,Total,229221.44,204726.01,433947.45,15,2025


In [34]:
total_entre_marcas = df_distribute_invest_copy[["brand_categoria", "original_cost"]]
total_entre_marcas = total_entre_marcas.iloc[:-1]
total_entre_marcas

,brand_categoria,original_cost
0,Bajaj,154861.19
1,Vento,13715.32
2,Honda,17842.59
3,Otros,23448.56
4,Italika,9690.80
5,TVS,9662.98


In [36]:
df_distribute_invest_copy = df_distribute_invest.copy()

total_entre_marcas = df_distribute_invest_copy[["brand_categoria", "original_cost"]]
total_entre_marcas = total_entre_marcas.iloc[:-1]
total_entre_marcas

costo_distribuido = df_distribute_invest_copy[["brand_categoria","distribute_cost", "total_cost"]]
costo_distribuido = costo_distribuido.iloc[:-1]

In [38]:
total_entre_marcas

,brand_categoria,original_cost
0,Bajaj,154861.19
1,Vento,13715.32
2,Honda,17842.59
3,Otros,23448.56
4,Italika,9690.80
5,TVS,9662.98


In [37]:
costo_distribuido

,brand_categoria,distribute_cost,total_cost
0,Bajaj,125110.33,279971.52
1,Vento,45494.67,59209.99
2,Honda,11373.67,29216.26
3,Otros,0.00,23448.56
4,Italika,11373.67,21064.47
5,TVS,11373.67,21036.65


In [41]:
total_entre_marcas

,brand_categoria,original_cost
0,Bajaj,154861.19
1,Vento,13715.32
2,Honda,17842.59
3,Otros,23448.56
4,Italika,9690.80
5,TVS,9662.98


In [60]:
invest_total_week = invest_total[invest_total["Semana"] == 15]

### Tabla inversión

In [61]:
# df_generic_invest_distribution
df_distribute_invest_copy = df_distribute_invest.copy()

total_entre_marcas = df_distribute_invest_copy[["brand_categoria", "original_cost"]]
total_entre_marcas = total_entre_marcas.iloc[:-1]
total_entre_marcas

#df df_distribute_invest
costo_distribuido = df_distribute_invest_copy[["brand_categoria","distribute_cost", "total_cost"]]
costo_distribuido = costo_distribuido.iloc[:-1]

inversion_total = costo_distribuido["total_cost"].sum()
display(costo_distribuido)
print(f"Inversión total: {inversion_total:.0f}")
print(f"Inversión total de la genérica: {generic_invest}")

# df: df_total_investment_plataforms_per_brand
invest_total_copy = invest_total_week.copy()
# Obtener las marcas que están en total_entre_marcas
marcas_a_excluir = total_entre_marcas["brand_categoria"].tolist()
# Filtrar el dataframe para excluir esas marcas
invest_total_copy = invest_total_copy[~invest_total_copy["Marca"].isin(marcas_a_excluir)]
invest_total_copy = invest_total_copy[invest_total_copy["Marca"] != "Generica"]
invest_total_copy = invest_total_copy.drop(columns = ["Semana", "Año"])

costo_distribuido
# df_distribute_invest
# Se elimina la columna de distribute_cost para sacar la tabla de Distribución PLP Desktop
costo_distribuido_copy = costo_distribuido.drop(columns = ["distribute_cost"])
# Se renombra para poder juntarlo con la tabla de las marcas que no tienen costos distribuidos
costo_distribuido_copy = costo_distribuido_copy.rename(columns={"brand_categoria": "Marca", "total_cost": "Cost"})

# Se concatenan junto con los costos distribuidos
tabla_inversion = pd.concat([costo_distribuido_copy, invest_total_copy])

,brand_categoria,distribute_cost,total_cost
0,Bajaj,125110.33,279971.52
1,Vento,45494.67,59209.99
2,Honda,11373.67,29216.26
3,Otros,0.00,23448.56
4,Italika,11373.67,21064.47
5,TVS,11373.67,21036.65


Inversión total: 433947
Inversión total de la genérica: 204726


In [68]:
tabla_inversion = tabla_inversion[tabla_inversion["Marca"] != "Otros"]

In [70]:
tabla_inversion

,Marca,Cost
0,Bajaj,279971.52
1,Vento,59209.99
2,Honda,29216.26
4,Italika,21064.47
5,TVS,21036.65
147,Yamaha,13920.50
96,Suzuki,8594.00
29,Dinamo,773.20
115,Treck,160.86


In [69]:
print(tabla_inversion["Cost"].sum())

433947.44999999995


In [222]:
df_generic = investment_analyzer.get_df_generic_invest_distribution()
tabla_final = process_investment_distribution(
    df_generic,
    invest_total_week,
    generic_invest
)

In [223]:
tabla_xd =  generar_tabla_final(tabla_inversion)

In [224]:
tabla_xd

,Marca,Cost
0,Bajaj,279971.52
1,Vento,59209.99
2,Honda,29216.26
3,Italika,21064.47
4,TVS,21036.65
5,Yamaha,13920.50
6,Suzuki,8594.00
7,CF Moto,0.00
8,Otro,934.06


In [226]:
print(tabla_xd["Cost"].sum())

433947.45


### CpRQL (Costo por RQL)

In [ ]:
# Calcular el costo por RQL para cada marca
df_costo_distribuido_entre_marcas_copy = df_costo_distribuido_entre_marcas.copy()
# Evitar división por cero
df_costo_distribuido_entre_marcas_copy['costo_por_rql'] = df_costo_distribuido_entre_marcas_copy.apply(
    lambda row: row['costo_distribuido'] / row['rql'] if row['rql'] > 0 else 0, axis=1
)
# Mostrar el resultado
display(df_costo_distribuido_entre_marcas_copy[['brand_invest', 'Semana', 'Año', 'costo_distribuido', 'rql', 'costo_por_rql']].sort_values(by='costo_por_rql', ascending=False))